In [2]:
import numpy as np     #只需要下载numpy库即可
import random
import GridWorld_v2
import time
from IPython.display import clear_output

In [3]:
gamma = 0.9   #折扣因子，越接近0越近视

rows = 5      #记得行数和列数这里要同步改
columns = 5

# gridworld = GridWorld_v2.GridWorld_v2(rows=rows, columns=columns, forbiddenAreaNums=8, targetNums=2, seed = 52,forbiddenAreaScore=-10)
# gridworld = GridWorld_v2.GridWorld_v2(desc = [".#",".T"])             #赵老师4-1的例子
# gridworld = GridWorld_v2.GridWorld_v2(desc = ["##.T","...#","...."])  #随便弄的例子
gridworld = GridWorld_v2.GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = [".....",".##..","..#..",".#T#.",".#..."]) 
#gridworld = GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = ["T."]) 
gridworld.show()


value = np.zeros(rows*columns)       #初始化可以任意，也可以全0
qtable = np.zeros((rows*columns,5))  #初始化，这里主要是初始化维数，里面的内容会被覆盖所以无所谓


⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️


In [4]:
def Expected_SARSA(gridworld:GridWorld_v2.GridWorld_v2,gamma = 0.99,trajectorySteps=-1, learning_rate=0.001, final_epsilon=0.01, num_episodes=600)->GridWorld_v2.GridWorld_v2:
    """
    这是Expected_SARSA算法

    Parameters:
    gamma (float): 近视远视程度
    trajectorySteps (int): 寻路的轨迹长度，如果是-1，则为寻到目的则停止，否则参数即为trajectory长度
    learning_rate (float): 学习率，用于调节TD-target
    epsilon (float): epsilon-greedy的核心参数，0~1的浮点数，其中1则表示当前state所有决策概率一样，0则表示决策没有任何的随机性
    num_episodes (int): 表示模型迭代次数

    Returns:
    GridWorld_v2.GridWorld_v2: 把模型返回回去
    """
    
    state_value = np.zeros((rows * columns))
    action_value = np.zeros((rows * columns, 5))
    policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))] 
    epsilon = 0.5
    for episode in range(num_episodes):
        #清除输出，可以更好的展示策略
        time.sleep(0.2)
        clear_output(wait=True)
        
        print("episode",f"{episode}/{num_episodes}")
        if(epsilon > final_epsilon) :
            epsilon -= 0.001
        else:
            epsilon = final_epsilon

        # p1是目标方向的概率，p0是另外四个方向的概率
        p1 = 1-epsilon * (4/5)
        p0 = epsilon/5
        d = {1:p1, 0:p0}
        # policy_epsilon是policy取epsilon-greedy的概率决策
        print("p1",p1,"p0",p0)
        policy_epsilon = np.vectorize(d.get)(policy)

        #cnt数组用来检查每个state有多少次访问
        cnt = [0 for i in range(25)]
        
        initState=10
        initAction=random.randint(0,4)  

        if trajectorySteps==-1:
            stop_when_reach_target = True
        Trajectory = gridworld.getTrajectoryScore(nowState=initState, 
                                                  action=initAction, 
                                                  policy=policy_epsilon, 
                                                  steps=trajectorySteps, 
                                                  stop_when_reach_target=True)
        Trajectory.append((17,4,1,17,4)) #让他最后自转一下
        print("trajectorySteps",len(Trajectory))
        

        
        # 注意这里的返回值是大小为(trajectorySteps+1)的元组列表，因为把第一个动作也加入进去了
        steps = len(Trajectory) - 1
        for k in range(steps,-1,-1):
            #State，Action，Reward，NextState，NextAction
            tmpstate, tmpaction, tmpscore, nextState, nextAction  = Trajectory[k]
            cnt[tmpstate] += 1
            #SARSA
            v = (action_value[nextState] * policy_epsilon[nextState]).sum()
            TD_error = action_value[tmpstate][tmpaction] - (tmpscore + gamma * v)
            action_value[tmpstate][tmpaction] -= learning_rate * TD_error

        # policy improvement
        policy = np.eye(5)[np.argmax(action_value,axis=1)]  #qtable的最优值作为更新策略，并用独热码来表示
        policy_epsilon = np.vectorize(d.get)(policy)
    
        #输出每个state的访问次数
        print(np.array(cnt).reshape(5,5)) 

        state_value = np.sum(policy_epsilon * action_value,axis=1)
        mean_state_value = np.sum(policy_epsilon * action_value,axis=1).mean()
        
        gridworld.showPolicy(policy)
        print(np.round(state_value,decimals=4).reshape(5,5))
        print("mean_state_value", mean_state_value)

    return gridworld
    

In [5]:
Expected_SARSA(gridworld)

episode 599/600
p1 0.992 p0 0.002
trajectorySteps 17
[[1 1 1 1 0]
 [1 0 0 1 0]
 [2 1 0 1 1]
 [0 0 2 0 1]
 [0 0 1 1 1]]
➡️➡️➡️⬇️⬇️
⬆️⏫️⏩️⬇️⬇️
⬆️⬅️⏬➡️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
[[-1.5800e-02 -8.6000e-03 -4.1000e-03 -2.4000e-03 -1.3000e-03]
 [-1.2900e-02 -3.8000e-03 -1.3000e-03 -2.0000e-03 -1.9000e-03]
 [-2.4600e-02 -2.7800e-02  1.0610e-01 -1.4000e-03 -1.9000e-03]
 [-2.7600e-02  2.0820e-01  1.0226e+00  4.5500e-02 -7.0000e-04]
 [-2.7400e-02  1.1200e-02  4.0850e-01  3.9900e-02  2.3000e-03]]
mean_state_value 0.06715079376971238
